In [28]:
import cv2
import numpy as np
import math

In [29]:
def findMaxContour(contours):
    max_i = 0
    max_area = 0

    for i in range(len(contours)):
        area_face = cv2.contourArea(contours[i])

        if max_area < area_face:
            max_area = area_face
            max_i = i

        try: 
            c = contours[max_i]

        except: 
            contours = [0]
            c = contours[0]

        return c

In [33]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if ret == False:
        break

    frame = cv2.resize(frame, (640, 480))
    frame = cv2.flip(frame, 1)

    roi = frame[100:200, 160:320]  # [y1:y2, x1:x2]
    cv2.rectangle(frame, (160, 100), (320, 200), (0, 0, 255), 0 )

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    lower_color = np.array([0, 45, 79], dtype=np.uint8)
    upper_color = np.array([17, 255, 255], dtype=np.uint8)

    mask = cv2.inRange(hsv, lower_color, upper_color)

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)
    mask = cv2.medianBlur(mask, 15)

    contours, _ = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        try:
            c = findMaxContour(contours)
            extremeLeft  = tuple(c[c[:, :, 0].argmin()][0])
            extremeRight = tuple(c[c[:, :, 0].argmax()][0])
            extremeTop   = tuple(c[c[:, :, 1].argmin()][0])
            extremeBot  = tuple(c[c[:, :, 0].argmax()][0])

            cv2.circle(roi, extremeLeft,    5, (0,255,0), 2)
            cv2.circle(roi, extremeRight,   5, (0,255,0), 2)
            cv2.circle(roi, extremeTop,     5, (0,255,0), 2)
            cv2.circle(roi, extremeBot,     5, (0,255,0), 2)

            cv2.line(roi, extremeLeft, extremeBot, (255,0,0), 2)
            cv2.line(roi, extremeBot, extremeRight, (255,0,0), 2)
            cv2.line(roi, extremeRight, extremeBot, (255,0,0), 2)
            cv2.line(roi, extremeBot, extremeLeft, (255,0,0), 2)

            topright = math.sqrt((extremeRight[0] - extremeTop[0]) ** 2 + (extremeRight[1] - extremeTop[1]) ** 2)
            
            rightbot = math.sqrt((extremeRight[0] - extremeBot[0]) ** 2 + (extremeRight[1] - extremeBot[1]) ** 2)
            
            botleft = math.sqrt((extremeLeft[0] - extremeBot[0]) ** 2 + (extremeLeft[1] - extremeBot[1]) ** 2)
            
            lefttop = math.sqrt((extremeLeft[0] - extremeTop[0]) ** 2 + (extremeLeft[1] - extremeTop[1]) ** 2)

            topbot = math.sqrt((extremeBot[0] - extremeTop[0]) ** 2 + (extremeBot[1] - extremeTop[1]) ** 2)
            
            leftright = math.sqrt((extremeLeft[0] - extremeRight[0]) ** 2 + (extremeLeft[1] - extremeRight[1]) ** 2)

            try:
                angle_trb =int( math.acos((topright**2 + rightbot**2 - topbot**2)/ (topright*rightbot*topbot)) * 57)
                cv2.putText(roi, str(angle_trb), (extremeRight[0]-10, extremeRight[1]), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255), 2, cv2.LINE_AA)

            except:
                cv2.putText(roi, "?", (extremeRight[0]-10, extremeRight[1]), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255), 2, cv2.LINE_AA)
            
        except:
            pass

    cv2.imshow("frame", frame)
    cv2.imshow("roi", roi)
    cv2.imshow("mask", mask)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [31]:
print(topright,rightbot,botleft, lefttop)        


14.212670403551895 0.0 14.212670403551895 0.0
